# Dependencies
***

Uncomment and run cell below to install CityLearn and other dependencies for the internal RL agents.

In [3]:
# # upgrade pip
!pip install --upgrade pip

# # Install dependencies
!pip install CityLearn scikit-learn==1.0.2 torch==1.12.0 torchvision==0.13.0

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\lmohsen\anaconda3\envs\scientificProject\python.exe -m pip install --upgrade pip


  Using cached scikit_learn-1.0.2-cp310-cp310-win_amd64.whl (7.2 MB)
  Using cached torchvision-0.13.0-cp310-cp310-win_amd64.whl (1.1 MB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached Pillow-9.2.0-cp310-cp310-win_amd64.whl (3.3 MB)
  Using cached gym-0.24.1-py3-none-any.whl
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0
  Rolling back uninstall of pillow
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\__init__.py
   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\__init__.py
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\__main__.py
   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\__main__.py
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\__pycache__\
   from C:\Users\lmohsen\anaconda3\envs\scientificProject\Lib\site

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'C:\\Users\\lmohsen\\anaconda3\\envs\\scientificProject\\Lib\\site-packages\\PIL\\msvcp140.dll'
Consider using the `--user` option or check the permissions.




   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\image.py
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\imagechops.py
   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\imagechops.py
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\imagecms.py
   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\imagecms.py
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\imagecolor.py
   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\imagecolor.py
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\imagedraw.py
   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\imagedraw.py
  Moving to c:\users\lmohsen\anaconda3\envs\scientificproject\lib\site-packages\pil\imagedraw2.py
   from C:\Users\lmohsen\AppData\Local\Temp\pip-uninstall-zhmhnxe6\imagedraw2.py
  Moving to c:\users\lmohsen\anaco

In [4]:
 # Import dependencies
import math
from pathlib import Path

import pandas as pd

from citylearn.agents.rbc import OptimizedRBC as RBCAgent
from citylearn.agents.sac import SACOptimizedRBC as SACAgent
from citylearn.citylearn import CityLearnEnv
from citylearn.cost_function import CostFunction

ImportError: cannot import name 'Image' from 'PIL' (C:\Users\lmohsen\anaconda3\envs\scientificProject\lib\site-packages\PIL\__init__.py)

# Helper Functions
***

In [ ]:
# Function to calculate KPIs
def get_kpis(env):
    ## building level
    building_kpis = []

    for b in env.buildings:
        building_kpis += [{
            'building_name': b.name,
            'kpi': 'electricity_consumption',
            'value': CostFunction.net_electricity_consumption(b.net_electricity_consumption)[-1]/\
                CostFunction.net_electricity_consumption(b.net_electricity_consumption_without_storage)[-1],
            }, {
            'building_name': b.name,
            'kpi': 'carbon_emissions',
            'value': CostFunction.carbon_emissions(b.net_electricity_consumption_emission)[-1]/\
                CostFunction.carbon_emissions(b.net_electricity_consumption_without_storage_emission)[-1],
            }, {
            'building_name': b.name,
            'kpi': 'zero_net_energy',
            'value': sum(b.net_electricity_consumption)/\
                sum(b.net_electricity_consumption_without_storage),
            }]

    building_kpis = pd.DataFrame(building_kpis)
    building_kpis['level'] = 'building'

    ## district level
    district_kpis = pd.DataFrame([{
        'kpi': 'ramping',
        'value': CostFunction.ramping(env.net_electricity_consumption)[-1]/\
            CostFunction.ramping(env.net_electricity_consumption_without_storage)[-1],
        }, {
        'kpi': '1 - load_factor',
        'value': CostFunction.load_factor(env.net_electricity_consumption)[-1]/\
            CostFunction.load_factor(env.net_electricity_consumption_without_storage)[-1],
        }, {
        'kpi': 'average_daily_peak',
        'value': CostFunction.average_daily_peak(env.net_electricity_consumption)[-1]/\
            CostFunction.average_daily_peak(env.net_electricity_consumption_without_storage)[-1],
        }, {
        'kpi': 'peak_demand',
        'value': CostFunction.peak_demand(env.net_electricity_consumption)[-1]/\
            CostFunction.peak_demand(env.net_electricity_consumption_without_storage)[-1],
        }])

    district_kpis = pd.concat([district_kpis, building_kpis], ignore_index=True, sort=False)
    district_kpis = district_kpis.groupby(['kpi'])[['value']].mean().reset_index()
    district_kpis['building_name'] = 'District'
    district_kpis['level'] = 'district'
    kpis = pd.concat([district_kpis, building_kpis], ignore_index=True, sort=False)

    return kpis

def parse_kpis(kpis):
    kpis['value'] = kpis['value'].round(3)
    kpis = kpis.pivot(index='kpi', columns='building_name', values='value')
    kpis = kpis.to_dict()
    kpis = {b: {kpi: v for kpi, v in b_kpis.items() if not math.isnan(v)} for b, b_kpis in kpis.items()}

    return kpis

# RBC Agent
***
Use the RBC agent to control the CityLearn Challenge 2022 Phase I environment.

In [2]:
schema = Path('../citylearn/data/citylearn_challenge_2022_phase_1/schema.json') # replace with applicable path to schema
env = CityLearnEnv(schema)
agents = RBCAgent(
    action_space=env.action_space,
    observation_space=env.observation_space,
    building_information=env.get_building_information(),
    observation_names=env.observation_names,
)
observations = env.reset()

while not env.done:
    actions = agents.select_actions(observations)

    # apply actions to env
    observations, rewards, _, _ = env.step(actions)

print('KPIs:', parse_kpis(get_kpis(env)))

NameError: name 'Path' is not defined

# SAC Agent
***
Use the SAC agent to control the CityLearn Challenge 2022 Phase I environment.

In [5]:
schema = Path('../citylearn/data/citylearn_challenge_2022_phase_1/schema.json') # replace with applicable path to schema
env = CityLearnEnv(schema)
agents = SACAgent(
    building_ids=[b.uid for b in env.buildings],
    action_space=env.action_space,
    observation_space=env.observation_space,
    building_information=env.get_building_information(),
    observation_names=env.observation_names,
    hidden_dimension=[256, 256],
    discount=0.99,
    tau=0.005,
    lr=0.003,
    batch_size=256,
    replay_buffer_capacity=1e5,
    start_training_time_step=6000,
    end_exploration_time_step=7000,
    deterministic_start_time_step=7500,
    action_scaling_coef=0.5,
    reward_scaling=5.0,
    update_per_time_step=2
)
episodes = 10 # number of training episodes

for e in range(episodes):
    observations = env.reset()

    while not env.done:
        actions = agents.select_actions(observations)

        # apply actions to env
        next_observations, rewards, _, _ = env.step(actions)

        # update agents
        agents.add_to_buffer(observations, actions, rewards, next_observations, done=env.done)
        observations = [o for o in next_observations]

    print('Episode:', e, '; KPIs:', parse_kpis(get_kpis(env)))

Episode: 0 ; KPIs: {'Building_1': {'carbon_emissions': 1.035, 'electricity_consumption': 1.057, 'zero_net_energy': 1.06}, 'Building_2': {'carbon_emissions': 1.067, 'electricity_consumption': 1.079, 'zero_net_energy': 1.041}, 'Building_3': {'carbon_emissions': 1.066, 'electricity_consumption': 1.089, 'zero_net_energy': 1.091}, 'Building_4': {'carbon_emissions': 1.062, 'electricity_consumption': 1.076, 'zero_net_energy': 1.032}, 'Building_5': {'carbon_emissions': 1.07, 'electricity_consumption': 1.098, 'zero_net_energy': 1.052}, 'District': {'1 - load_factor': 0.99, 'average_daily_peak': 1.021, 'carbon_emissions': 1.06, 'electricity_consumption': 1.08, 'peak_demand': 0.992, 'ramping': 1.128, 'zero_net_energy': 1.055}}
Episode: 1 ; KPIs: {'Building_1': {'carbon_emissions': 1.009, 'electricity_consumption': 1.011, 'zero_net_energy': 1.104}, 'Building_2': {'carbon_emissions': 1.073, 'electricity_consumption': 1.074, 'zero_net_energy': 1.078}, 'Building_3': {'carbon_emissions': 1.185, 'elect